In [46]:
from sklearn.ensemble import RandomForestClassifier
import file_tagger
import sequence_genrator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



In [47]:
GYRO = False
SEQUENCE_LENGTH = 4
SEQUENCE_OVERLAP = 3

ACTIVITIES_TO_CLASSIFY = {

    "lying_down_left&coughing",
    "lying_down_left&hyperventilating",
    "lying_down_left&talking",
    "lying_down_left&singing",
    "lying_down_left&laughing",
  
}

DATA_DIRECTORY = "./all_respeck"


In [48]:
# generate training data
def generate_training_data(directory, sequence_length, overlap, gyro = GYRO): # if gyro is false, only accelerometer data is used

    tagged_data = []

    # group each csv file into their respective areas
    csv_dictionary = file_tagger.tag_directory(directory)

    # iterates through each activity
    for key in ACTIVITIES_TO_CLASSIFY:

        # iterates through each csv file for the activity 
        for csv_file in csv_dictionary[key]:
            if gyro:
                sequences = sequence_genrator.generate_sequences_from_file_with_gyroscope(directory + "/" + csv_file, sequence_length, overlap)
            else:
                sequences = sequence_genrator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequence_length, overlap)

            # iterate through each generated sequence
            for sequence in sequences:
                tagged_data.append((key, sequence))

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")
    return tagged_data

In [49]:

tagged_data = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, gyro = GYRO)
X = [sequence.flatten() for tag, sequence in tagged_data]
y = [tag for tag, sequence in tagged_data]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Create a Random Forest classifier with 100 trees (you can adjust the parameters as needed)
rf_classifier = RandomForestClassifier(n_estimators=200)

# Fit the Random Forest to the training data
rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest Accuracy: {accuracy}")


there are 6201 tagged sequences in the dataset
Random Forest Accuracy: 0.8420628525382756
